# LangChain's components

## Models and Prompts

In [39]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:3b",temperature=0)
# llm = ChatOllama(model="deepseek-r1:1.5b",temperature=0)
model=llm

In [40]:
from langchain_core.messages import HumanMessage

response=model.invoke([HumanMessage(content="Tell Me a Joke")])

In [41]:
print(response.content)

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you smile! Do you want to hear another one?


In [42]:
from langchain import PromptTemplate

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "the cat is on the table", language = "spanish"))

Sentence: the cat is on the table
Translation in spanish:


In [43]:
import csv

# Sample data
data = [
    ['Name', 'Age', 'City'],
    ['John', 25, 'New York'],
    ['Emily', 28, 'Los Angeles'],
    ['Michael', 22, 'Chicago']
]

# File name
file_name = 'sample.csv'

# Write data to CSV file
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'Sample CSV file "{file_name}" generated and saved.')

Sample CSV file "sample.csv" generated and saved.


In [44]:

# Sample sentences about mountains and nature
content = """Amidst the serene landscape, towering mountains stand as majestic guardians of nature's beauty.
The crisp mountain air carries whispers of tranquility, while the rustling leaves compose a symphony of wilderness.
Nature's palette paints the mountains with hues of green and brown, creating an awe-inspiring sight to behold.
As the sun rises, it casts a golden glow on the mountain peaks, illuminating a world untouched and wild."""

# File name
file_name = 'mountain.txt'

# Write content to text file
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

#print(f'Sample text file "{file_name}" generated and saved.')


with open('mountain.txt') as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(

    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])
print(texts[0])
print(texts[1])
print(texts[2])

page_content='Amidst the serene landscape, towering mountains stand as majestic guardians of nature's beauty.'
page_content='The crisp mountain air carries whispers of tranquility, while the rustling leaves compose a'
page_content='leaves compose a symphony of wilderness.'


## Text embedding models

In [45]:
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="llama3")
# embeddings_model = OllamaEmbeddings(model="deepseek-r1")
embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)

print("Embed documents:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

print("Embed query:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")

Embed documents:
Number of vector: 5; Dimension of each vector: 4096
Embed query:
Dimension of the vector: 4096
Sample of the first 5 elements of the vector: [0.0029812825, -0.018019853, 0.019897195, -0.0028292192, -0.00059465313]


In [46]:
#saving the conversation in a txt file
# List of dialogue lines
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]

# File name
file_name = 'dialogue.txt'

# Write dialogue lines to text file
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'Dialogue text file "{file_name}" generated and saved.')

Dialogue text file "dialogue.txt" generated and saved.


## Vector stores

In [47]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [48]:
from langchain.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [49]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.

raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
embeddings = OllamaEmbeddings(model="llama3")
db = FAISS.from_documents(documents, embeddings)

In [50]:
query = "What is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Sorry to hear that. May I ask your name?


# Memory

In [52]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

# llm = ChatOllama(model="llama3.2:3b",temperature=0)

memory = ConversationSummaryMemory(llm=ChatOllama(model="llama3.2:3b",temperature=0))
memory.save_context({"input": "hi, I'm looking for some ideas to write an essay in AI"}, {"output": "hello, what about writing on LLMs?"})

memory.load_memory_variables({})

/tmp/ipykernel_7747/887915033.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=ChatOllama(model="llama3.2:3b",temperature=0))


{'history': 'The human asks for ideas to write an essay using AI. The AI suggests writing about Large Language Models (LLMs).'}

# Chains

In [53]:
from langchain import PromptTemplate, LLMChain

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = ChatOllama(model="llama3.2:3b",temperature=0)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict(sentence="the cat is on the table", language="spanish")

/tmp/ipykernel_7747/1309620874.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


'La gata está en la mesa.'

## Router chain

In [54]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = ChatOllama(model="llama3.2:3b",temperature=0)

itinerary_template = """You are a vacation itinerary assistant. \
You help customers finding the best destinations and itinerary. \
You help customer screating an optimized itinerary based on their preferences.

Here is a question:
{input}"""

restaurant_template = """You are a restaurant booking assitant. \
You check with customers number of guests and food preferences. \
You pay attention whether there are special conditions to take into account.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "itinerary",
        "description": "Good for creating itinerary",
        "prompt_template": itinerary_template,
    },
    {
        "name": "restaurant",
        "description": "Good for help customers booking at restaurant",
        "prompt_template": restaurant_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)


/tmp/ipykernel_7747/3838304983.py:44: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm=llm, output_key="text")
/home/thinkpalm/Documents/My Projects/LangChain_projects/.env/lib/python3.11/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/tmp/ipykernel_7747/3838304983.py:56: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [55]:
print(chain.run("I want to book a table for tonight"))

/tmp/ipykernel_7747/3374663280.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run("I want to book a table for tonight"))




> Entering new MultiPromptChain chain...
restaurant: {'input': "I'd like to book a table for dinner tonight"}
> Finished chain.
I'd be happy to help you book a table for dinner tonight.

Before I get started, can you please tell me how many guests will be joining you? And do you have any dietary restrictions or preferences, such as vegetarian, gluten-free, or any other specific requirements?

Also, would you like to dine in our main restaurant or perhaps one of our private dining rooms?


## Sequential Chain

In [56]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = ChatOllama(model="llama3.2:3b",temperature=0)

template = """You are a comedian. Generate a joke on the following {topic}
Joke:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """You are translator. Given a text input, translate it to {language}
Translation:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)


In [57]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("Cats and Dogs")



> Entering new SimpleSequentialChain chain...
Why did the cat join a band?

Because it wanted to be the purr-cussionist! (get it?)
Why did the cat join a band?
Because it wanted to be the purr-cussionist! (get it?)

> Finished chain.


## Transformation chain

In [58]:
# Import the string module
import string

# Define the function
def rename_cat(inputs: dict) -> dict:
  # Open the file in read mode
  text = inputs["text"]
  # Create a table that maps punctuation characters to None
  new_text = text.replace('cat', 'Silvester the Cat')
  # Apply the table to the text and return the result
  return {"output_text": new_text}

In [60]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

with open("Cats&Dogs.txt") as f:
    cats_and_dogs = f.read()


import string

llm = ChatOllama(model="llama3.2:3b",temperature=0)

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=rename_cat
)

template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

sequential_chain.run(cats_and_dogs)

"Here is a summary of the text:\n\nA cat named Silvester and a dog lived in the same house but didn't get along. One day, Silvester played a prank on the dog by tying yarn around his tail, leading to a chaotic chase. The dog eventually caught up with Silvester and they fought back and forth until their owner intervened. After being scolded and cleaned up, both animals apologized and promised to be nicer to each other. From then on, they became friends and learned to appreciate each other's differences, playing, sleeping, and eating together in harmony."